# Titanic: Machine Learning from Disaster

In [1]:
# https://www.kaggle.com/c/titanic

In [2]:
import sklearn
import pandas as pd

In [506]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [489]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [508]:

test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [509]:
s=pd.concat([train,test])
s.Survived.value_counts()

0.0    549
1.0    342
Name: Survived, dtype: int64

In [510]:
s_dummies = pd.get_dummies(s, columns=['Sex', 'Pclass', 'Embarked'])
#test_dummies = pd.get_dummies(test, columns=['Sex', 'Pclass', 'Embarked'])

In [511]:
s_dummies.head()

,Age,Cabin,Fare,Name,Parch,PassengerId,SibSp,Survived,Ticket,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,22.0,NaN,7.2500,"Braund, Mr. Owen Harris",0,1,1,0.0,A/5 21171,0,1,0,0,1,0,0,1
1,38.0,C85,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,1.0,PC 17599,1,0,1,0,0,1,0,0
2,26.0,NaN,7.9250,"Heikkinen, Miss. Laina",0,3,0,1.0,STON/O2. 3101282,1,0,0,0,1,0,0,1
3,35.0,C123,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,1.0,113803,1,0,1,0,0,0,0,1
4,35.0,NaN,8.0500,"Allen, Mr. William Henry",0,5,0,0.0,373450,0,1,0,0,1,0,0,1


## Preprocess data

In [512]:
s_dummies_new=s_dummies.drop(['Name', 'Ticket', 'Cabin'], axis=1)
s_dummies_new.head()

,Age,Fare,Parch,PassengerId,SibSp,Survived,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,1,1,0.0,0,1,0,0,1,0,0,1
1,38.0,71.2833,0,2,1,1.0,1,0,1,0,0,1,0,0
2,26.0,7.9250,0,3,0,1.0,1,0,0,0,1,0,0,1
3,35.0,53.1000,0,4,1,1.0,1,0,1,0,0,0,0,1
4,35.0,8.0500,0,5,0,0.0,0,1,0,0,1,0,0,1


In [513]:
# Extract features

X_t = s_dummies_new.drop(['Survived'], axis=1)
#X_test=test_dummies.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
#X_test.head()
for_test=X_t.PassengerId[X_t['PassengerId']>=892]
for_test.head()

0    892
1    893
2    894
3    895
4    896
Name: PassengerId, dtype: int64

In [567]:
# Extract targets
y_train = s_dummies_new[('Survived')].dropna()
y_train=y_train.apply(lambda x: int(x))
y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [517]:
# Fill NA values
from sklearn.preprocessing import Imputer
# Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=0, copy=True)
imp=Imputer(missing_values="NaN", strategy="mean",axis=0)
imp.fit(X_ed)
X_ed_imp=imp.fit_transform(X_ed).ravel()
X_ed_imp

array([ 22.  ,   7.25,   0.  , ...,   1.  ,   0.  ,   0.  ])

In [518]:
# Scale features
from sklearn.preprocessing import StandardScaler
import numpy as np
# StandardScaler(copy=True, with_mean=True, with_std=True)
scaler = StandardScaler()
scaler.fit(X_ed_imp)
X_scaled = scaler.transform(X_ed_imp)
# Fit logistic regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
X_scaled=X_scaled.reshape(1309,-1)
#x=680*891
#X_scaled_train = X_scaled[:x]
X_scaled_train = X_scaled[:891]
X_scaled_test = X_scaled[-418:]
len(X_scaled_test)

/Users/annaarova/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/annaarova/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


418

In [516]:
X_ed=X_t.drop(['PassengerId'], axis=1)
#X_train = X_t[X_t.test_train==1].drop(['PassengerId'], axis=1)
#X_t['Age'].apply(lambda x: x if x!='NaN'else x=mean(x))
#lambda x: (x-min(x))/(max(x)-min(x))
#X_train.drop(['test_train'], axis=1, inplace=True)
X_ed.head()

,Age,Fare,Parch,SibSp,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,1,0,1,0,0,1,0,0,1
1,38.0,71.2833,0,1,1,0,1,0,0,1,0,0
2,26.0,7.9250,0,0,1,0,0,0,1,0,0,1
3,35.0,53.1000,0,1,1,0,1,0,0,0,0,1
4,35.0,8.0500,0,0,0,1,0,0,1,0,0,1


## Train model and make predictions

In [568]:
# LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, 
#                    fit_intercept=True, intercept_scaling=1, class_weight=None, 
#                    random_state=None, solver='liblinear', max_iter=100,
#                    multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)
model = RandomForestClassifier().fit(X_scaled_train, y_train)

k_fold = KFold(n=10, shuffle=True, random_state=100)

In [569]:
from sklearn.metrics import make_scorer
scores = cross_val_score(RandomForestClassifier(),
                         X_scaled_train, y_train, 
                         scoring = make_scorer(accuracy_score),
                         cv = k_fold)
pipeline = Pipeline(steps=[
                           ('scale', scaler),
                           ('model', RandomForestClassifier())])
from sklearn.grid_search import GridSearchCV
param = {
    
    'model__max_depth': (3,4,5,6,7,8,9,10,12),
    'model__max_features' :(5,7,10,11,12)
}

grid = GridSearchCV(pipeline, param, 
                    scoring=make_scorer(accuracy_score),
                    cv=k_fold)
model = grid.fit(X_scaled_train, y_train)

In [570]:
model.get_params().keys()

dict_keys(['estimator__steps', 'estimator__model__min_weight_fraction_leaf', 'param_grid', 'estimator__model__min_samples_leaf', 'verbose', 'estimator__model__max_leaf_nodes', 'error_score', 'estimator__model__max_features', 'estimator__model__n_estimators', 'pre_dispatch', 'n_jobs', 'estimator__model__n_jobs', 'iid', 'estimator__model__warm_start', 'estimator__scale__with_mean', 'cv', 'estimator__model__bootstrap', 'estimator__model__class_weight', 'estimator__scale__copy', 'refit', 'estimator__model__min_samples_split', 'estimator__scale', 'estimator__scale__with_std', 'estimator__model__max_depth', 'estimator__model__random_state', 'estimator__model__verbose', 'estimator__model__oob_score', 'estimator__model', 'estimator', 'fit_params', 'scoring', 'estimator__model__criterion'])

In [571]:
(model.best_params_, model.best_score_)

({'model__max_depth': 3, 'model__max_features': 5}, 1.0)

## Estimate quality

In [572]:
# Estimate quality
from sklearn.metrics import accuracy_score
# accuracy_score(y_true, y_pred, normalize=True, sample_weight=None)
predicted = grid.best_estimator_.predict(X_scaled_test)
predicted[:5]

array([0, 1, 0, 0, 0])

## Create submission 

In [573]:
with open('submission_finality.txt', 'w') as out:
    out.write('PassengerId,Survived\n')
    for passenger, y in zip(for_test, predicted):
        out.write('%s,%s\n' % (passenger, y))